In [ ]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 4.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

import category_encoders as ce
from google.colab import drive

# Preparation

In [ ]:
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/My Drive/dataset/kidney_disease')
df = data.copy()
df

Mounted at /content/drive


,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,395,55.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,...,47,6700,4.9,no,no,no,good,no,no,notckd
396,396,42.0,70.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,...,54,7800,6.2,no,no,no,good,no,no,notckd
397,397,12.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,...,49,6600,5.4,no,no,no,good,no,no,notckd
398,398,17.0,60.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,...,51,7200,5.9,no,no,no,good,no,no,notckd


In [ ]:
test = pd.get_dummies(df)
print(test)

      id   age    bp     sg   al   su    bgr    bu   sc    sod  ...  cad_yes  \
0      0  48.0  80.0  1.020  1.0  0.0  121.0  36.0  1.2    NaN  ...        0   
1      1   7.0  50.0  1.020  4.0  0.0    NaN  18.0  0.8    NaN  ...        0   
2      2  62.0  80.0  1.010  2.0  3.0  423.0  53.0  1.8    NaN  ...        0   
3      3  48.0  70.0  1.005  4.0  0.0  117.0  56.0  3.8  111.0  ...        0   
4      4  51.0  80.0  1.010  2.0  0.0  106.0  26.0  1.4    NaN  ...        0   
..   ...   ...   ...    ...  ...  ...    ...   ...  ...    ...  ...      ...   
395  395  55.0  80.0  1.020  0.0  0.0  140.0  49.0  0.5  150.0  ...        0   
396  396  42.0  70.0  1.025  0.0  0.0   75.0  31.0  1.2  141.0  ...        0   
397  397  12.0  80.0  1.020  0.0  0.0  100.0  26.0  0.6  137.0  ...        0   
398  398  17.0  60.0  1.025  0.0  0.0  114.0  50.0  1.0  135.0  ...        0   
399  399  58.0  80.0  1.025  0.0  0.0  131.0  18.0  1.1  141.0  ...        0   

     appet_good  appet_poor  pe_no  pe_

In [ ]:
df.shape[1]

26

In [ ]:
df.isna().sum() / df.shape[0]*100

id                 0.00
age                2.25
bp                 3.00
sg                11.75
al                11.50
su                12.25
rbc               38.00
pc                16.25
pcc                1.00
ba                 1.00
bgr               11.00
bu                 4.75
sc                 4.25
sod               21.75
pot               22.00
hemo              13.00
pcv               17.50
wc                26.25
rc                32.50
htn                0.50
dm                 0.50
cad                0.50
appet              0.25
pe                 0.25
ane                0.25
classification     0.00
dtype: float64

#preprocessing

In [ ]:
df = df.drop(['rc', 'rbc', 'id'],axis=1)

In [ ]:
df.head()

,age,bp,sg,al,su,pc,pcc,ba,bgr,bu,...,hemo,pcv,wc,htn,dm,cad,appet,pe,ane,classification
0,48.0,80.0,1.020,1.0,0.0,normal,notpresent,notpresent,121.0,36.0,...,15.4,44,7800,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,normal,notpresent,notpresent,NaN,18.0,...,11.3,38,6000,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,notpresent,notpresent,423.0,53.0,...,9.6,31,7500,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,abnormal,present,notpresent,117.0,56.0,...,11.2,32,6700,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,notpresent,notpresent,106.0,26.0,...,11.6,35,7300,no,no,no,good,no,no,ckd


In [ ]:
# olah = df.iloc[:, 1:22]

In [ ]:
# missing data
for i in df.columns:
  if df[i].dtype == object:
    df[i].fillna(df[i].mode()[0], inplace=True)
  else:
    df[i].fillna(df[i].mean(), inplace=True)

In [ ]:
df.isna().sum() / df.shape[0] * 100

age               0.0
bp                0.0
sg                0.0
al                0.0
su                0.0
pc                0.0
pcc               0.0
ba                0.0
bgr               0.0
bu                0.0
sc                0.0
sod               0.0
pot               0.0
hemo              0.0
pcv               0.0
wc                0.0
htn               0.0
dm                0.0
cad               0.0
appet             0.0
pe                0.0
ane               0.0
classification    0.0
dtype: float64

In [ ]:
encoder = ce.OrdinalEncoder(df)

In [ ]:
# df1 = df
df = encoder.fit_transform(df)

In [ ]:
df.head()

,age,bp,sg,al,su,pc,pcc,ba,bgr,bu,...,hemo,pcv,wc,htn,dm,cad,appet,pe,ane,classification
0,48.0,80.0,1.020,1.0,0.0,normal,notpresent,notpresent,121.000000,36.0,...,15.4,44,7800,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,normal,notpresent,notpresent,148.036517,18.0,...,11.3,38,6000,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,notpresent,notpresent,423.000000,53.0,...,9.6,31,7500,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,abnormal,present,notpresent,117.000000,56.0,...,11.2,32,6700,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,notpresent,notpresent,106.000000,26.0,...,11.6,35,7300,no,no,no,good,no,no,ckd


In [ ]:
# jalankan jika menggunakan iqr
olah = df.iloc[:, df.columns != 'age']

# IQR

In [ ]:
for x in olah.columns:
  # setup iqr
  q1 = df1[x].quantile(0.25)
  q3 = df1[x].quantile(0.75)
  iqr = q3 - q1
  lower_limit = q1 - (iqr * 1.5)
  upper_limit = q1 + (iqr * 1.5)
  # replace outliers
  df1.loc[df1[x] < lower_limit, x] = lower_limit
  df1.loc[df1[x] > upper_limit, x] = upper_limit

NameError: ignored

In [ ]:
olah['age']=df1['age']

NameError: ignored

In [ ]:
olah

#KNN sklearn

In [ ]:
X = df1.iloc[:, df1.columns != 'classification']
y = df1.iloc[:, df1.columns == 'classification']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

NameError: ignored

In [ ]:
from sklearn import metrics
# model = KNeighborsClassifier(n_neighbors=11).fit(X_train,y_train)
# model.fit(X_train, y_train)
# print('Train Accuracy',model.score(X_train, y_train))

knn = KNeighborsClassifier(n_neighbors = 11).fit(X_train,y_train)
yhat= knn.predict(X_test)
print(metrics.accuracy_score(y_test, yhat))

In [ ]:
# y_test = np.array(y_test).reset_index()

In [ ]:
yhat = pd.DataFrame(yhat)

# Confussion manual

In [ ]:
confussion = pd.concat([y_test, yhat], axis=1)
confussion.columns = ['real', 'pred']

In [ ]:
confussion['pred'].unique()

In [ ]:
a = confussion.pivot_table(index="real", columns='pred', aggfunc=np.count_nonzero)/2
a[2] = a[1]
a[2] = 0
cols = a.columns.tolist()
cols = [1, 2, 3]
a = a[cols]
a = a.fillna(0)
a
# a.sum().sum()

In [ ]:
for i in range(3):
  print(i, i, i)

In [ ]:
b = 0
for i in range(3):
  b = b + a.iloc[i, i]
b/(a.sum().sum())

# convusion

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred=knn.predict(X_test)
y_true=y_test

cm=confusion_matrix(y_true,y_pred)
cm

In [ ]:
f, ax=plt.subplots(figsize=(5,5))
sns.heatmap(cm,annot=True,linewidths=0.5,linecolor="red",fmt=".0f",ax=ax)
plt.xlabel("y_pred")
plt.ylabel("y_true")
plt.show()

test

In [ ]:
from sklearn import metrics
mean_acc = np.zeros(20)
for i in range(1,21):
    #Train Model and Predict
    knn = KNeighborsClassifier(n_neighbors = i).fit(X_train,y_train)
    yhat= knn.predict(X_test)
    mean_acc[i-1] = metrics.accuracy_score(y_test, yhat)

mean_acc

# split

In [ ]:
k1 = df[df['classification']==1]
k2 = df[df['classification']==2]
k3 = df[df['classification']==3]

In [ ]:
print('total data :',k1.shape)
print('total data :',k2.shape)
print('total data :',k3.shape)

total data : (248, 23)
total data : (2, 23)
total data : (150, 23)


In [ ]:
k1

,age,bp,sg,al,su,pc,pcc,ba,bgr,bu,...,hemo,pcv,wc,htn,dm,cad,appet,pe,ane,classification
0,48.0,80.0,1.020000,1.000000,0.000000,1,1,1,121.000000,36.0,...,15.400000,1,1,1,1,1,1,1,1,1
1,7.0,50.0,1.020000,4.000000,0.000000,1,1,1,148.036517,18.0,...,11.300000,2,2,2,2,1,1,1,1,1
2,62.0,80.0,1.010000,2.000000,3.000000,1,1,1,423.000000,53.0,...,9.600000,3,3,2,1,1,2,1,2,1
3,48.0,70.0,1.005000,4.000000,0.000000,2,2,1,117.000000,56.0,...,11.200000,4,4,1,2,1,2,2,2,1
4,51.0,80.0,1.010000,2.000000,0.000000,1,1,1,106.000000,26.0,...,11.600000,5,5,2,2,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,48.0,100.0,1.017408,1.016949,0.450142,1,1,1,103.000000,79.0,...,6.300000,30,30,1,2,2,2,1,1,1
246,48.0,110.0,1.015000,3.000000,0.000000,1,2,1,106.000000,215.0,...,8.600000,32,56,1,2,2,1,1,2,1
247,54.0,90.0,1.025000,1.000000,0.000000,2,1,1,150.000000,18.0,...,12.526437,11,6,2,2,1,2,2,2,1
248,59.0,70.0,1.010000,1.000000,3.000000,2,1,1,424.000000,55.0,...,12.600000,14,51,1,1,2,1,1,1,1


klasifiaksi 1


In [ ]:
train_1=k1.sample(frac=0.8,random_state=200)
test_1=k1.drop(train_1.index)

In [ ]:
X_train1 = train_1.iloc[:, train_1.columns != 'classification']
y_train1 = train_1.iloc[:, train_1.columns == 'classification']

X_test1 = test_1.iloc[:, test_1.columns != 'classification']
y_test1 = test_1.iloc[:, test_1.columns == 'classification']

In [ ]:
X_train1 =train_1.iloc[:,train_1.columns != 'classification']
y_train1 =train_1.iloc[:,train_1.columns == 'classification']

X_test1 = test_1.iloc[:, test_1.columns != 'classification']
y_test1 = test_1.iloc[:, test_1.columns == 'classification']

In [ ]:
X_train1

,age,bp,sg,al,su,pc,pcc,ba,bgr,bu,...,pot,hemo,pcv,wc,htn,dm,cad,appet,pe,ane
145,57.000000,90.0,1.015000,5.000000,0.000000,2,1,2,148.036517,322.0,...,4.800000,8.000000,13,61,1,1,2,2,2,2
183,30.000000,70.0,1.015000,0.000000,0.000000,1,1,1,101.000000,106.0,...,4.300000,12.526437,11,6,2,2,1,2,1,1
22,48.000000,80.0,1.025000,4.000000,0.000000,2,1,1,95.000000,163.0,...,3.800000,9.800000,4,7,1,2,1,1,1,2
191,51.483376,70.0,1.010000,3.000000,0.000000,1,1,1,110.000000,115.0,...,2.700000,9.100000,32,16,1,1,1,2,1,1
131,5.000000,50.0,1.010000,0.000000,0.000000,1,1,1,148.036517,25.0,...,4.627244,11.800000,7,58,2,2,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,56.000000,70.0,1.015000,4.000000,1.000000,1,1,1,210.000000,26.0,...,3.800000,16.100000,22,41,2,2,1,1,1,1
181,45.000000,70.0,1.025000,2.000000,0.000000,2,2,1,117.000000,52.0,...,3.800000,10.000000,15,69,2,2,1,1,1,1
204,65.000000,90.0,1.010000,4.000000,2.000000,1,1,1,172.000000,82.0,...,6.300000,8.800000,3,6,1,1,1,1,2,2
162,59.000000,70.0,1.017408,1.016949,0.450142,1,1,1,204.000000,34.0,...,4.100000,9.800000,14,2,2,1,1,1,1,1


klasifiaksi 2

In [ ]:
train_2=k2.sample(frac=0.5,random_state=200)
test_2=k2.drop(train_2.index)

In [ ]:
X_train2 = train_1.iloc[:, train_2.columns != 'classification']
y_train2 = train_1.iloc[:, train_2.columns == 'classification']

X_test2 = test_2.iloc[:, test_2.columns != 'classification']
y_test2 = test_2.iloc[:, test_2.columns == 'classification']

In [ ]:
X_train2 =train_2.iloc[:,train_2.columns != 'classification']
y_train2 =train_2.iloc[:,train_2.columns == 'classification']

X_test2 = test_2.iloc[:, test_2.columns != 'classification']
y_test2 = test_2.iloc[:, test_2.columns == 'classification']

klasifikasi 3

In [ ]:
train_3=k3.sample(frac=0.8,random_state=200)
test_3=k3.drop(train_3.index)

In [ ]:
X_train3 = train_3.iloc[:, train_3.columns != 'classification']
y_train3 = train_3.iloc[:, train_3.columns == 'classification']

X_test3 = test_3.iloc[:, test_3.columns != 'classification']
y_test3 = test_3.iloc[:, test_3.columns == 'classification']

In [ ]:
X_train3 =train_3.iloc[:,train_3.columns != 'classification']
y_train3 =train_3.iloc[:,train_3.columns == 'classification']

X_test3 = test_3.iloc[:, test_3.columns != 'classification']
y_test3 = test_3.iloc[:, test_3.columns == 'classification']

In [ ]:
X_train = pd.concat([X_train1, X_train2, X_train3]).values.tolist()
X_test = pd.concat([X_test1, X_test2, X_test3]).values.tolist()

y_train = pd.concat([y_train1, y_train2, y_train3]).values.tolist()
y_test = pd.concat([y_test1, y_test2, y_test3]).values.tolist()

In [ ]:
# np.unique(y_test['classification'], return_counts=True)

In [ ]:
X_train.info()

In [ ]:
# np.unique(y_train['classification'], return_counts=True)

# Model

In [ ]:
def jarak(a, b, p = 2):
    # store dimension number
    dim = len(a)
    # set initial distance to 0
    distance = 0
    # calulate distance
    for i in range(dim):
        distance += abs((a[i]) - int(b[i]))**p
    distance = distance**(1/p)
    print(a[i])
    return distance

# test
# jarak(a = X.iloc[0], b = X.iloc[1], p = 2)

In [ ]:
def knn_predict(X_train, X_test, y_train, y_test, k, p):
    # make predictions on test data
    # need output of 1 prediction per test data point
    y_hat_test = []
    for test_point in X_test:
        distances = []

        for train_point in X_train:
            distance = jarak(test_point, train_point, p = p)
            distances.append(distance)

        # store distances in dataframe
        df_dists = pd.DataFrame(data = distances, columns = ['dist'], index = y_train.index)
        # store distances and only consider the k closest points
        df_nn = df_dists.sort_values(by = ['dist'], axis = 0)[:k]
        # create counter object to track the labels of k closest points
        counter = Counter(y_train[df_nn.index])
        # get most common label of all nearest neighbors
        prediction = counter.most_common()[0][0]
        # append prediction to output list
        y_hat_test.append(prediction)

    return y_hat_test

# make predictions on test dataset
y_hat_test = knn_predict(X_train, X_test, y_train, y_test, k = 5, p = 2)
print(y_hat_test)